# Energy Price Analysis and Forecasting

This notebook demonstrates how to work with the energy price forecasting model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import xgboost as xgb
from prepare_features import (
    create_time_features,
    create_holiday_features,
    create_lag_features,
    create_target_features,
    prepare_features_for_training
)
from utils import (
    calculate_metrics,
    plot_feature_importance,
    plot_predictions,
    plot_error_distribution,
    TimeSeriesCV
)

# Set plotting style
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## Load and Explore the Data

In [ ]:
# Load raw prices
raw_prices = pd.read_csv('data/raw_prices.csv')
raw_prices['timestamp'] = pd.to_datetime(raw_prices['timestamp'])
raw_prices.set_index('timestamp', inplace=True)

# Display basic information
print("Dataset Info:")
print(f"Time range: {raw_prices.index.min()} to {raw_prices.index.max()}")
print(f"Number of records: {len(raw_prices)}")
print("\nSample data:")
raw_prices.head()

## Load Pre-processed Features

In [ ]:
# Load preprocessed features
features = pd.read_csv('data/features_scaled.csv')
features['timestamp'] = pd.to_datetime(features['timestamp'])
features.set_index('timestamp', inplace=True)

print("Available features:")
for col in features.columns:
    print(f"- {col}")

## Create Custom Features

You can create additional features using the provided functions:

In [ ]:
# Example: Create custom time features
custom_time_features = create_time_features(raw_prices)
custom_time_features.head()

## Load and Use Trained Models

In [ ]:
# Load a model for a specific horizon (e.g., 1-hour ahead)
horizon = 1
model = xgb.XGBRegressor()
model.load_model(f'data/models/model_h{horizon}.json')

# Get feature importance
feature_cols = [col for col in features.columns if not col.startswith('target_')]
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
})

# Plot feature importance
plot_feature_importance(importance, title=f'Feature Importance (t+{horizon} horizon)')

## Make Predictions

In [ ]:
# Get the last 48 hours of data
recent_data = features.tail(48)

# Make predictions
X = recent_data[feature_cols]
y_true = recent_data[f'target_t{horizon}']
y_pred = model.predict(X)

# Plot predictions
plt.figure(figsize=(15, 6))
plt.plot(y_true.index, y_true.values, label='Actual', alpha=0.7)
plt.plot(y_true.index, y_pred, label='Predicted', alpha=0.7)
plt.title(f'Actual vs Predicted Prices (t+{horizon} horizon)')
plt.xlabel('Time')
plt.ylabel('Price (€/MWh)')
plt.legend()
plt.grid(True)
plt.show()

# Print metrics
metrics = calculate_metrics(y_true, y_pred)
print("\nPrediction Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

## Experiment with New Features

You can create new features and retrain the model:

In [ ]:
def create_custom_features(df):
    """Add your custom features here"""
    features = pd.DataFrame(index=df.index)
    
    # Example: Add price momentum features
    features['price_momentum_3h'] = df['price_eur_per_mwh'].pct_change(3)
    features['price_momentum_6h'] = df['price_eur_per_mwh'].pct_change(6)
    
    return features

# Create and view new features
custom_features = create_custom_features(raw_prices)
custom_features.head()